In [ ]:
import pandas as pd
from glob import glob

In [ ]:
json_filenames = glob('../data/json/*.jsonl')
print(json_filenames)

In [ ]:
json_file_path = '../data/dataset_autos_V2.jsonl'
df = pd.read_json(json_file_path, lines=True)

In [ ]:
df['id'] = df['id'].astype(str)
df['slug'] = df['slug'].astype(str)
df['sucursal'] = df['sucursal'].astype("category")
df['caja'] = df['caja'].astype("category")
df['details'] = df['details'].astype("string")
df['year'] = df['year'].astype("Int64")
df['km'] = df['km'].astype("Int64")

df_sorted = df.sort_values('km', na_position='last')

df_final = df_sorted.drop_duplicates(subset=['id'], keep='first')
df_final.to_json('../data/dataset_autos_V2_cleaned.jsonl', orient='records', lines=True)

In [ ]:

csv_path = '../data/csv/data_2025_12_30-02h_13m.csv'
df_to_clean = pd.read_csv(csv_path, encoding='utf-8')

conteo_registros = df_to_clean['ID_Auto'].value_counts()
ids_duplicados = conteo_registros[conteo_registros > 6]

id_ejemplo = ids_duplicados.index[0]

df_limpio = df_to_clean.drop_duplicates()
df_limpio['ID_Auto'].nunique()
df_limpio.to_csv('../data/csv/data_2025_12_30-02h_13m_cleaned.csv', index=False, encoding='utf-8')